# [Магические методы](https://docs.python.org/3/reference/datamodel.html#special-method-names)

**Магические методы** (*Dunder Methods* - от словосочетания **d**ouble **under**score) в Python – это специальные методы, которые начинаются и заканчиваются двумя символами подчеркивания, например `__init__` или `__str__`. Магические методы позволяют переопределять или расширять поведение встроенных операторов и функций для пользовательских классов и объектов. Например, вы можете использовать магические методы для определения того, как с объектами будут работать операторы `+`, `<`, `in` и др., или как объекты будут выводятся на печать.

Неплохой гайд по магическим методам представлен в этой [статье](https://habr.com/ru/articles/186608/).

Список магических методов представлен [здесь](https://mathspp.com/blog/pydonts/dunder-methods).


## Методы перехвата доступа к полям объекта

Следующии методы вызываются при взаимодействиях с полями объектов.

- `__setattr__()` – при создании или изменении поля;
- `__getattribute__()` – при получении значения поля (как существующего, так и несуществующего);
- `__getattr__()` – при попытке получении значения несуществующего поля;
- `__delattr__()` – при удалении поля.

При переопределении этих методов (кроме `__getattr__()`), следует также вызывать одноименный метода суперкласса (или класса `object`, но в этом случае первым аргументом ему нужно передать `self`).

Как и всем методам объектов первым позиционным аргументом в метод будет передан сам объект, а вторым аргументом – имя поля, с которым происходит взаимодействие. Метод `__setattr__()` получает помимо этого еще и третий аргумент – значение, которое будет присвоено полю.

> Изменение поля внутри метода `__setattr__()` приведет к повторному вызову метода, внутри которого будет снова изменение поля и снова вызов, т.е. бесконечная рекурсия.

В общем случае если поле не определено, его можно добавлять динамически Определим поле класса `fields`, в котором будут содержаться имена допустимых аттрибутов, для того, чтобы нельзя было добавлять новые аттрибуты. В методе `__setattr__()` генерируется исключение `AttributeError`, когда возникает попытка присвоить значение несуществующему полю:

In [ ]:
class Rectangle:

    fields = ('width', 'height')

    def __init__(self, width, height):
        self.width = width
        self.height = height

    def __setattr__(self, name, value):
        if not name in self.fields:
            raise AttributeError('attribute is not exist')
        else:
            super().__setattr__(name, value)

    def __delattr__(self, name):
        print(f'delete attribute {name}')
        object.__delattr__(self, name)

In [ ]:
r = Rectangle(3, 4)

## Создание и удаление объекта

`__new__()` является *методом класса* и вызывается перед созданием нового экземпляра класса, и соответственно, до инициализации (т.е. до вызова метода `__init__()`) и принимает первым аргументом сам класс (`cls`), за которым следуют аргументы, передаваемые в `__init__()`. Также методу передаются аргументы, получаемые в конструкторе, поэтому можно задать универсальные аргументы `*args`, `**kwargs`, либо соответствующими аргументам в `__init__()`.

Метод должен возвращать экземпляр класса, поэтому в `return` стоит вызвать одноименный метод суперкласса и передать ему ссылку на класс (передавать другие аргументы не нужно, если это не предусмотрено в наследуемом классе).

Возвращаемый экземпляр еще не инициализирован, поэтому на этом этапе `__dict__` этого объекта пока пуст. Метод переопределяется в крайне ограниченных случаях.

`__del__()` (финализатор) вызывается перед удалением (оператором `del` либо сборщиком мусора) экземпляра класса. В рассматриваемом примере при удалении экземпляра мы хотим менять поле класса, поэтому для обращения к полю `counter` класса используется выражение `type(self)`, которое возвращает класс объекта, у которого вызван метод `__del__()`.

In [ ]:
class Rectangle:

    counter = 0     # поле класса, которое будет хранить число объектов класса:

    def __new__(cls, *args, **kwargs):
        print(f'{ args   = }, { kwargs = }')
        cls.counter += 1
        return super().__new__(cls)

    def __init__(self, a, b) -> None:
        self.a, self.b = a, b

    def __del__(self):
        type(self).counter -= 1


s1 = Rectangle(1, 2)
s2 = Rectangle(3, b=4)
s3 = Rectangle(a=5, b=6)
print(f"{Rectangle.counter = }")
del s1
print(f"{Rectangle.counter = }")

In [ ]:
class AltInt(int):
    def __new__(cls, value):
        print(f"{ value = }")
        return super().__new__(cls, value * 10)

v = AltInt(5)

В качестве еще одного примера реализуем класс, у которого может существовать только один экземпляр в приложении. Такие объекты называются **одиночками** (*singleton*). Для этого определим поле класса `_instance` которое инициализируется значением `None`. При создании первого экземпляра класса в методе `__new__()` полю класса `_instance` будет присвоен сам объект (возвращаемый методом `__new__()` суперкласса). При последующих попытках создания нового объекта класса, будет возврщаться созданный уже экземпляр класса.

In [ ]:
class Singleton:
    _instance = None            # Class attribute to store the single instance

    def __new__(cls):
        if cls._instance is None:       # Check if there is no instance
            # Create and store the instance:
            cls._instance = super().__new__(cls)
        return cls._instance            # Return the existing instance

s1 = Singleton()
s2 = Singleton()
s1 is s2

## Поведение объекта

Благодаря модели данных в Python пользовательские типы могут вести себя так же естественно, как встроенные. Для этого достаточно просто реализовать методы, необходимые для того, чтобы объект вел себя ожидаемым образом. Можно опеределить магические методы, которые определяют поведение объекта при взаимодействии со встроенными функциями, такими как `len()` или `abs()`, операторами, такими как `+`, `>`, `in`. Также можно определить методы, позволяющие работать с элементами объекта посредством передачи индксов или ключей в квадратных скобках `[]`, подобно тому, как это делается в списках и словарях. Можно определить поведение объекта при преобразовании типов.

In [1]:
class Rectangle():
    def __init__(self, length=1, width=1):
        self.length, self.width = length, width

    def get_area(self):
        return self.length * self.width

    def get_perimeter(self):
        return self.length*2 + self.width*2

    def __repr__(self):
        return f"Rectangle(lenght={self.length}, width={self.width})"

    def __str__(self):
        return f"Rectangle {self.length}x{self.width} with Area = {self.get_area()}"

    def __len__(self):
        return self.get_perimeter()

    def __bool__(self):
        return bool(self.get_area())


### `__repr__()`

При выполнении последней строки ячейки с кодом, если строка представляет собой выражение, то на печать выводится объект, возвращаемый выражением. Фактически, вызывается метод `__repr__()` объекта, и на печать выводится то, что возвращается этим методом. Это значение также может быть получено и при вызове встроенной функции `repr()`. Метод `__repr__()` должен возвращать строку, которая представляет информацию об объекте (`representation`), и желательно, чтобы эта информация могла быть полезна для разработчика при отладке.

In [3]:
r = Rectangle(2, 3)
r                       # будет вызван метод __repr__()

Rectangle(lenght=2, width=3)

In [10]:
repr(r)

'<__main__.Rectangle object at 0x000001AA59173110>'

In [6]:
a = str(r)

In [4]:
len(r)

10

In [6]:
r.__str__()

'Rectangle 2x3 with Area = 6'

In [7]:
a

'Rectangle 2x3 with Area = 6'

### `__str__()`

При вызове функции `print()`, функция вызывает метод `__str__()`, который должен возвращать строкое представление объекта. В отличие от `__repr__()` здесь в приоритете читабельность - краткое и понятное описание объекта для конечного пользователя. Метод `__str__()` также вызывается при конвертации в тип `str`.

>Если метод `__str__()` не определен в классе объекта, то `print()` и `str()` будут использовать метод `__repr__()`. Обратное, однако, неверно: `repr()` не станет вызывать метод `__str__()`, если метод `__repr__()` не определен.

In [11]:
print(r)

Rectangle 2x3 with Area = 6


In [ ]:

str(r)

NameError: name 'r' is not defined

Метод `__len__()` определяет возвращаемое функцией `len()` значение, при передаче объекта в функцию:

In [ ]:
len(r)

### Преобразование типов

Метод `__bool__()` определяет поведение объекта при конвертации в булевый тип.
По умолчанию любой объект пользовательского класса считается истинным, но положение меняется, если реализован хотя бы один из методов `__bool__()` или `__len__()`. Функция `bool(x)`, по существу, вызывает `x.__bool__()` и использует полученный результат. Если метод `__bool__()` не реализован, то Python пытается вызвать `x.__len__()` и при получении нуля функция `bool()` возвращает `False`. В противном случае `bool()` возвращает `True`.

Также можно определить поведение объекта при преобразовании в другие типы `__int__()`, `__float__()` и т.д.

In [ ]:
bool(r)

### Операторы

Для того, чтобы переопределить поведение объекта с **операторами сравнения**, такие как `==`, `!=`, `<` и тд. Следует переопределять такие методы как `__eq__()`, `__ne__()`, `__lt__()` и [другие](https://habr.com/ru/articles/186608/#comparisons).

Для переопределения бинарных **арифметических операторов** (`+`, `-`, `*`...) используются такие методы как `__add__()` `__sub__()` `__mul__()` и т.д. Эти методы срабатывают когда объект находится слева от оператора и  принимают в качестве аргумента операнд, расположеный справа от оператора. Для того, чтобы определить поведение объекта в том случае, когда объект расположен справа от оператора, следует переопределить такие методы как `__radd__()` `__rsub__()` `__rmul__()` и т.д. (приставка `r` означает *right*).

Для **унарных операторов** `-`, `+` используются методы `__neg__()`, `__pos__()`, для функции `abs()`, соответственно, метод  `__abs__()`.

Для арифметических **операторов присоения** `+=`: `__iadd__()`, `-=`: `__isub__()`, `*=`: `__imul__()`, `/=`: `__itruediv__()`, `//=` `__ifloordiv__()`, `%=`: `__imod__()`, `**=`: `__ipow__()`.

Для переопределения **операторов принадлежности** `in` и `not in` следует переопределять метод `__contains__()`.

Также можно определить методы и для **поразрядных операторов**.